<a href="https://colab.research.google.com/github/skywalker0803r/Ricky/blob/master/%E7%8E%89%E5%B1%B1%E9%8A%80%E8%A1%8CAI%E6%AF%94%E8%B3%BDEDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 中文map

In [0]:
中文map = {'bacno':'歸戶帳號','txkey':'交易序號','locdt':'授權日期','loctm':'授權時間','cano':'交易卡號',
         'contp':'交易類別','etymd':'交易型態','mchno':'特店代號','acqic':'收單行代碼','mcc':'MCC_CODE',
         'conam':'交易金額-台幣(經過轉換)','ecfg':'網路交易註記','insfg':'分期交易註記','iterm':'分期期數',
         'stocn':'消費地國別','scity':'消費城市','stscd':'狀態碼','ovrlt':'超額註記碼','flbmk':'Fallback註記',
         'hcefg':'支付型態','csmcu':'消費地幣別','flg_3dsmk':'3DS交易註記','fraud_ind':'盜刷註記'}

# train data

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
train = pd.read_csv("/content/drive/My Drive/玉山人工智慧比賽數據/train.csv")
print(train.shape)
train.head()

(1521787, 23)


,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,fraud_ind,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6881,113261,38038,513.80,5,0,N,0,N,N,0,5,N,0,33,172652.0,457,59333,N,0,102,0,516056
1,0,134508,45725,465.62,5,0,N,2,N,N,0,0,N,0,9,105114.0,451,0,N,5817,102,0,4376
2,6881,15408,188328,513.80,5,0,N,0,N,N,0,5,N,0,6,152458.0,457,59333,N,0,102,0,483434
3,6716,157159,29967,1016.11,5,62,N,5,N,N,0,5,N,0,5,172946.0,247,50436,N,3281,102,0,1407164
4,5975,105985,81305,713.66,5,62,N,4,N,N,0,5,N,0,6,182129.0,263,93775,N,5817,102,0,1051004


# test data

In [4]:
test = pd.read_csv("/content/drive/My Drive/玉山人工智慧比賽數據/test.csv")
print(test.shape)
test_txkey = test["txkey"]
test.head()

(421665, 22)


,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6881,163188,116799,513.8,5,0,N,0,N,N,5,N,0,102,215328.0,457,59360,N,0,102,0,592489
1,6881,163188,116799,513.8,5,0,N,0,N,N,5,N,0,102,222007.0,457,59360,N,0,102,0,592452
2,6881,163188,116799,513.8,5,0,N,0,N,N,5,N,0,100,170013.0,457,59360,N,0,102,0,590212
3,6881,163188,116799,513.8,5,0,N,0,N,N,5,N,0,100,165914.0,457,59360,N,0,102,0,590209
4,6881,163188,116799,513.8,5,0,N,0,N,N,5,N,0,102,215311.0,457,59360,N,0,102,0,592488


# 缺失值情形

In [0]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [6]:
print(missing_values_table(train))
print(missing_values_table(test))

Your selected dataframe has 23 columns.
There are 2 columns that have missing values.
           Missing Values  % of Total Values
flbmk               12581                0.8
flg_3dsmk           12581                0.8
Your selected dataframe has 22 columns.
There are 2 columns that have missing values.
           Missing Values  % of Total Values
flbmk                3715                0.9
flg_3dsmk            3715                0.9


# 處理缺失值 fillna

把缺失值當成一種'unknown'類別

In [7]:
train.fillna('unknown',inplace=True)
test.fillna('unknown',inplace=True)
print(train.isnull().sum().sum())
print(test.isnull().sum().sum())

0
0


# Label encoding

In [8]:
from sklearn.preprocessing import LabelEncoder
# Create a label encoder object
le = LabelEncoder()
le_count = 0
le_col = []

# Iterate through the columns
for col in train.drop(['txkey','fraud_ind'],axis=1).columns:
  # 條件
  if len(train[col].unique())<=3:
    # Train on the training data
    le.fit(train[col])
    # Transform both training and testing data
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])
    # Keep track of how many columns were label encoded
    le_count += 1
    le_col.append(col)
            
print('%d columns were label encoded.' % le_count)
print(le_col)

5 columns were label encoded.
['ecfg', 'flbmk', 'flg_3dsmk', 'insfg', 'ovrlt']


## float col do Label encoding

In [9]:
float_col = []
for col in train.columns:
  if train[col].dtype == 'float':
    float_col.append(col)
print(float_col)

['conam', 'loctm']


In [0]:
for col in float_col:
  train[col] = train[col].astype(int)
  test[col] = test[col].astype(int)

In [11]:
train.head()

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,fraud_ind,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6881,113261,38038,513,5,0,0,0,0,0,0,5,0,0,33,172652,457,59333,0,0,102,0,516056
1,0,134508,45725,465,5,0,0,2,0,0,0,0,0,0,9,105114,451,0,0,5817,102,0,4376
2,6881,15408,188328,513,5,0,0,0,0,0,0,5,0,0,6,152458,457,59333,0,0,102,0,483434
3,6716,157159,29967,1016,5,62,0,5,0,0,0,5,0,0,5,172946,247,50436,0,3281,102,0,1407164
4,5975,105985,81305,713,5,62,0,4,0,0,0,5,0,0,6,182129,263,93775,0,5817,102,0,1051004


In [12]:
train.dtypes

acqic        int64
bacno        int64
cano         int64
conam        int64
contp        int64
csmcu        int64
ecfg         int64
etymd        int64
flbmk        int64
flg_3dsmk    int64
fraud_ind    int64
hcefg        int64
insfg        int64
iterm        int64
locdt        int64
loctm        int64
mcc          int64
mchno        int64
ovrlt        int64
scity        int64
stocn        int64
stscd        int64
txkey        int64
dtype: object

In [13]:
test.head()

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6881,163188,116799,513,5,0,0,0,0,0,5,0,0,102,215328,457,59360,0,0,102,0,592489
1,6881,163188,116799,513,5,0,0,0,0,0,5,0,0,102,222007,457,59360,0,0,102,0,592452
2,6881,163188,116799,513,5,0,0,0,0,0,5,0,0,100,170013,457,59360,0,0,102,0,590212
3,6881,163188,116799,513,5,0,0,0,0,0,5,0,0,100,165914,457,59360,0,0,102,0,590209
4,6881,163188,116799,513,5,0,0,0,0,0,5,0,0,102,215311,457,59360,0,0,102,0,592488


In [14]:
test.dtypes

acqic        int64
bacno        int64
cano         int64
conam        int64
contp        int64
csmcu        int64
ecfg         int64
etymd        int64
flbmk        int64
flg_3dsmk    int64
hcefg        int64
insfg        int64
iterm        int64
locdt        int64
loctm        int64
mcc          int64
mchno        int64
ovrlt        int64
scity        int64
stocn        int64
stscd        int64
txkey        int64
dtype: object

# one hot

In [0]:
#one_hot_col = []

#for col in train.drop(['fraud_ind','txkey','locdt']+le_col,axis=1).columns:
#  len_u = len(train[col].unique())
#  if len_u < 200:
#    one_hot_col.append(col)
#print(one_hot_col)

#train = pd.get_dummies(train,columns=one_hot_col)
#test = pd.get_dummies(test,columns=one_hot_col)

# Aligning Training and Testing Data

In [16]:
print('Training Features shape: ', train.shape)
print('Testing Features shape: ', test.shape)

train_labels = train['fraud_ind']

# Align the training and testing data, keep only columns present in both dataframes
train.drop('fraud_ind',axis=1,inplace=True)
train, test = train.align(test, join = 'inner', axis = 1)

# Add the target back in
train['fraud_ind'] = train_labels

print('Training Features shape: ', train.shape)
print('Testing Features shape: ', test.shape)

Training Features shape:  (1521787, 23)
Testing Features shape:  (421665, 22)
Training Features shape:  (1521787, 23)
Testing Features shape:  (421665, 22)


# after data preprocessing save data 

In [17]:
train.to_csv("/content/drive/My Drive/玉山人工智慧比賽數據/train_預處理完.csv")
test.to_csv("/content/drive/My Drive/玉山人工智慧比賽數據/test_預處理完.csv")
print('save done!')

save done!
